In [45]:
import numpy as np
import pandas as pd
import re
import warnings
from scipy import stats
# Interactive plotting with FigureWidget ipywidgets
from plotly import graph_objs as go
from ipywidgets import widgets

In [46]:
# Represent probabilities of a tier hit
level1 = [1.0, 0, 0, 0, 0]
level2 = [1.0, 0, 0, 0, 0]
level3 = [.65, .3, .5, 0, 0]
level4 = [.5, .35, .15, 0, 0]
level5 = [.37, .35, .25, .03, 0]
level6 = [.245, .35, .3, .10, .005]
level7 = [.2, .3, .33, .15, .02]
level8 = [.15, .25, .35, .2, .05]
level9 = [.1, .15, .35, .3, .1]
levels_df = pd.DataFrame([level1,level2,level3,level4,level5,level6,level7,level8,level9],
                      columns=['Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5'])

# Represent tier numbers & frequencies
tier1 = (13, 39)
tier2 = (13, 26)
tier3 = (13, 21)
tier4 = (10, 13)
tier5 = (6, 10)
tiers_df = pd.DataFrame([tier1,tier2,tier3,tier4,tier5], columns=['Unique Characters', 'Characters in Pool'])

# Establish color dict for tiers
#color_dict = {0:'Red', 1:'Gray', 2:'Green', 3:'Blue', 4:'Purple', 5:'Gold'}
color_dict = {0:'Red',1:'#A0A094',2:'#2DAE5E',3:'#3EBDE6',4:'#B375F2',5:'#F2B425',10:'Brown'}

# State current patch that has these characteristics
patch = '9.16b'

In [60]:
#TODO: Make opinionated toggle that changes total characters available in a pool dependent on current level
#TODO: Make Dash web version
#TODO: Implement median version
def hit_prob(level, tier, levels_df=levels_df, tiers_df=tiers_df, removed=0, others_claimed=0):
    tier = int(tier)
    if (1 <= tier <= 5) and (1 <= level <= 9):
        tier_prob = float(levels_df.loc[level-1,'Tier {}'.format(tier)])
        hits_in_pool = int(tiers_df.loc[tier-1,'Characters in Pool'])
        if removed > hits_in_pool: 
            removed = hits_in_pool
        char_hits = hits_in_pool - removed
        chars_in_tier = int(tiers_df.loc[tier-1,'Unique Characters'])
        if others_claimed > (chars_in_tier-1)*hits_in_pool:
            others_claimed = (chars_in_tier-1)*hits_in_pool
        hit_prob_solo = (char_hits/((chars_in_tier-1)*hits_in_pool + char_hits - others_claimed))*tier_prob
        hit_prob = hit_prob_solo*5
        return hit_prob
    else:
        return 0.0

def multi_hits_prob(level, tiers, levels_df=levels_df, tiers_df=tiers_df, removed=0, others_claimed=0):
    multi_hits_prob = 0
    if isinstance(tiers,list):
        for tier in tiers:
            multi_hits_prob += hit_prob(level, tier, levels_df, tiers_df, removed, others_claimed)
    else:
        mult_hits_prob = hit_prob(level, tiers, levels_df, tiers_df, removed, others_claimed)
    return multi_hits_prob

def multi_rolls_prob(level, tiers, rolls=10, levels_df=levels_df, tiers_df=tiers_df, removed=0, others_claimed=0):
    multi_rolls_prob = list(map(lambda x:(x+1)*multi_hits_prob(level, tiers, 
                            levels_df, tiers_df, removed, others_claimed), list(range(rolls))))
    return multi_rolls_prob

def custom_bar(level,rolls,champ,champ_letter):
    complete_bar = go.Bar(x=list(range(1,rolls+1)), 
              y=multi_rolls_prob(level, [int(champ)], rolls=rolls),
              name='Champion {}'.format(champ_letter),
              text=['{0:.4f}'.format(x) for x in multi_rolls_prob(level, [int(champ)], rolls=rolls)],
              textposition='auto',
              hoverinfo='y+name',
              marker={'color':color_dict[int(champ)],
                     'line': {'width':1.5}
                     }
             )
    return complete_bar

def custom_scatter(level,rolls,tier,champ_letter):
    prob = hit_prob(level, int(tier))
    custom_scatter = go.Scatter(x=list(range(1,rolls+1)),
                               y=stats.geom.cdf(list(range(1,rolls+1)),prob),
                                name='Champion {0}, median = {1}'.format(champ_letter, stats.geom.median(prob)),
                                text=['{0:.4f}'.format(x) for x in stats.geom.cdf(list(range(1,rolls+1)),prob)],
                                hoverinfo='y+name',
                               marker={'color':color_dict[int(tier)]})
    return custom_scatter

def champ_widget(initial_value,champ_letter):
    complete_champ = widgets.Text(
        value=str(initial_value),
        description='Champion {}'.format(champ_letter),
        layout={'width':'155px'})
    return complete_champ
 
def cdf_version(rolls,prob):
    return stats.geom.cdf(list(range(1,rolls+1)),prob)

# TFTdex display

In [69]:
initial_level = 1
initial_rolls = 10

level_widget = widgets.BoundedIntText(
    value=initial_level,
    min=1,
    max=9,
    step=1,
    description='Current Level',
    layout={'width':'155px'}
    )

rolls_widget = widgets.BoundedIntText(
    value=initial_rolls,
    min=1,
    max=30,
    step=1,
    description='Max Rolls',
    layout={'width':'155px'}
    )

champA = champ_widget(1,'A')
champB = champ_widget(0,'B')
champC = champ_widget(0,'C')
champD = champ_widget(0,'D')
champE = champ_widget(0,'E')

level_rolls_container = widgets.HBox([level_widget,rolls_widget])
champ_container = widgets.HBox(children=[champA, champB, champC, champD, champE])

In [70]:
#TODO: make df of y values to make more readable
#TODO: make first 1.0 line a solid black line
#TODO: put in median on geo plot
#TODO: shrinker marker sizes on geo plot

barA = custom_bar(level_widget.value,initial_rolls,champA.value,'A')
barB = custom_bar(level_widget.value,initial_rolls,champB.value,'B')
barC = custom_bar(level_widget.value,initial_rolls,champC.value,'C')
barD = custom_bar(level_widget.value,initial_rolls,champD.value,'D')
barE = custom_bar(level_widget.value,initial_rolls,champE.value,'E')

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    scattA = custom_scatter(level_widget.value,initial_rolls,champA.value,'A')
    scattB = custom_scatter(level_widget.value,initial_rolls,champB.value,'B')
    scattC = custom_scatter(level_widget.value,initial_rolls,champC.value,'C')
    scattD = custom_scatter(level_widget.value,initial_rolls,champD.value,'D')
    scattE = custom_scatter(level_widget.value,initial_rolls,champE.value,'E')    
    tot_prob = hit_prob(level_widget.value,champA.value)
    scattTot = go.Scatter(x=list(range(1,initial_rolls+1)),
                                   y=stats.geom.cdf(list(range(1,initial_rolls+1)),tot_prob),
                                    name='Total, median = {}'.format(stats.geom.median(tot_prob)),
                                    text=['{0:.4f}'.format(x) for x in stats.geom.cdf(
                                        list(range(1,initial_rolls+1)),tot_prob)],
                                    hoverinfo='y+name',
                                   marker={'color':color_dict[10]})

fig = go.FigureWidget(data=[barE, barD, barC, barB, barA],
             layout=go.Layout(
                 title='TFTdex (Patch {})'.format(patch),
                 height=500,
                 barmode='stack',
                 yaxis={'range':[0,6],
                   'title':'Expected number of hits',
                   'nticks':8,
                    'hoverformat':'.4f'},
                 ygrid={''}
                 xaxis={'title':'Number of Rolls',
                       'dtick':1},
                 legend={'traceorder':'reversed'}
             ))

scattfig = go.FigureWidget(data=[scattE, scattD, scattC, scattB, scattA, scattTot],
             layout=go.Layout(
                 title='Cumulative Distribution Function',
                 height=500,
                 yaxis={'range':[0,1],
                        'title':'Likelihood of at least one hit',
                        'nticks':10,
                       'hoverformat':'.4f'},
                 xaxis={'title':'Number of Rolls',
                       'dtick':1},
                 legend={'traceorder':'reversed'}
             ))


In [71]:
def validate():
    if 1 <= level_widget.value <= 9:
        return True
    else:
        return False
    
def scatt_update(level, rolls, letter, champ_df):
    if letter=='total':
        prob = sum(champ_df['hit_prob'])
        prob_dist = stats.geom.cdf(list(range(1,rolls+1)),prob)
        letter_idx = 5
        tier = 10
        scattfig.data[letter_idx].name='Total, median = {}'.format(stats.geom.median(prob))    
    else:
        idx_dict = {'A':4,'B':3,'C':2,'D':1,'E':0}
        letter_idx = idx_dict[letter]
        tier = champ_df.loc[letter,'champ_tier']
        removed = champ_df.loc[letter,'removed']
        others_claimed = champ_df.loc[letter,'others_claimed']
        prob = hit_prob(level, tier, removed=removed, others_claimed=others_claimed)
        prob_dist = stats.geom.cdf(list(range(1,rolls+1)),prob)
        scattfig.data[letter_idx].name='Champion {0}, median = {1}'.format(letter, stats.geom.median(prob))  

    scattfig.data[letter_idx].x=list(range(1,rolls+1))
    scattfig.data[letter_idx].y=prob_dist
    scattfig.data[letter_idx].text=['{0:.4f}'.format(x) for x in prob_dist]
    scattfig.data[letter_idx].marker['color']=color_dict.get(tier,'Black')
      


def check(change):
    if validate:
        level = level_widget.value
        rolls = rolls_widget.value
        champ_df = pd.DataFrame(columns=['input_string','split_string','champ_tier','removed','others_claimed','hit_prob'],
                          index=['A','B','C','D','E']).fillna(0)
        champ_df['input_string'] = [champA.value, champB.value, champC.value, champD.value, champE.value]
        champ_df['split_string'] = [list(filter(None, re.sub('[^,0-9]','',x).split(','))) for x in champ_df['input_string']]
        champ_df['champ_tier'] = [int(x[0]) if x else 0 for x in champ_df['split_string']]
        champ_df['removed'] = [int(x[1]) if len(x)>1 else 0 for x in champ_df['split_string']]
        champ_df['others_claimed'] = [int(x[2]) if len(x)>2 else 0 for x in champ_df['split_string']]
        champ_df['hit_prob'] = [hit_prob(level,champ_df.loc[x,'champ_tier'],
                                         removed=champ_df.loc[x,'removed'],
                                         others_claimed=champ_df.loc[x,'others_claimed']) for x in champ_df.index]
        
#TODO: make df of prob values to make more extensible
        with fig.batch_update():
            
            fig.data[4].x=list(range(1,rolls+1))
            fig.data[4].y=multi_rolls_prob(level, [champ_df.loc['A','champ_tier']], rolls=rolls, 
                                           removed=champ_df.loc['A','removed'], 
                                           others_claimed=champ_df.loc['A','others_claimed'])
            fig.data[4].text=['{0:.4f}'.format(x) for x in multi_rolls_prob(level, rolls=rolls,
                                            tiers=[champ_df.loc['A','champ_tier']], 
                                           removed=champ_df.loc['A','removed'],
                             others_claimed=champ_df.loc['A','others_claimed'])]
            fig.data[4].marker['color']=color_dict.get(champ_df.loc['A','champ_tier'],'Black')
            fig.data[3].x=list(range(1,rolls+1))
            fig.data[3].y=multi_rolls_prob(level, [champ_df.loc['B','champ_tier']], rolls=rolls,
                                           removed=champ_df.loc['B','removed'],
                                          others_claimed=champ_df.loc['B','others_claimed'])
            fig.data[3].text=['{0:.4f}'.format(x) for x in multi_rolls_prob(level, rolls=rolls,
                                            tiers=[champ_df.loc['B','champ_tier']], 
                                           removed=champ_df.loc['B','removed'],
                             others_claimed=champ_df.loc['B','others_claimed'])]
            fig.data[3].marker['color']=color_dict.get(champ_df.loc['B','champ_tier'],'Black')
            fig.data[2].x=list(range(1,rolls+1))
            fig.data[2].y=multi_rolls_prob(level, [champ_df.loc['C','champ_tier']], rolls=rolls,
                                           removed=champ_df.loc['C','removed'],
                                          others_claimed=champ_df.loc['C','others_claimed'])
            fig.data[2].text=['{0:.4f}'.format(x) for x in multi_rolls_prob(level, rolls=rolls,
                                            tiers=[champ_df.loc['C','champ_tier']], 
                                           removed=champ_df.loc['C','removed'],
                             others_claimed=champ_df.loc['C','others_claimed'])]
            fig.data[2].marker['color']=color_dict.get(champ_df.loc['C','champ_tier'],'Black')
            fig.data[1].x=list(range(1,rolls+1))
            fig.data[1].y=multi_rolls_prob(level, [champ_df.loc['D','champ_tier']], rolls=rolls,
                                           removed=champ_df.loc['D','removed'],
                                          others_claimed=champ_df.loc['D','others_claimed'])
            fig.data[1].text=['{0:.4f}'.format(x) for x in multi_rolls_prob(level, rolls=rolls,
                                            tiers=[champ_df.loc['D','champ_tier']], 
                                           removed=champ_df.loc['D','removed'],
                             others_claimed=champ_df.loc['D','others_claimed'])]
            fig.data[1].marker['color']=color_dict.get(champ_df.loc['D','champ_tier'],'Black')
            fig.data[0].x=list(range(1,rolls+1))
            fig.data[0].y=multi_rolls_prob(level, [champ_df.loc['E','champ_tier']], rolls=rolls,
                                           removed=champ_df.loc['E','removed'],
                                          others_claimed=champ_df.loc['E','others_claimed'])
            fig.data[0].text=['{0:.4f}'.format(x) for x in multi_rolls_prob(level, rolls=rolls,
                                            tiers=[champ_df.loc['E','champ_tier']], 
                                           removed=champ_df.loc['E','removed'],
                             others_claimed=champ_df.loc['E','others_claimed'])]
            fig.data[0].marker['color']=color_dict.get(champ_df.loc['E','champ_tier'],'Black')
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                scatt_update(level,rolls,'A', champ_df)
                scatt_update(level,rolls,'B', champ_df)
                scatt_update(level,rolls,'C', champ_df)
                scatt_update(level,rolls,'D', champ_df)
                scatt_update(level,rolls,'E', champ_df)
                scatt_update(level,rolls,'total', champ_df)

level_widget.observe(check, names='value')
rolls_widget.observe(check, names='value')
champA.observe(check, names='value')
champB.observe(check, names='value')
champC.observe(check, names='value')
champD.observe(check, names='value')
champE.observe(check, names='value')

In [72]:
print('Input desired champion info in the format "TIER,NUMBER OF CLAIMED CHAMPS,NUMBER OF OTHER CLAIMED CHAMPS IN TIER"')
full_plot = widgets.VBox([fig, level_rolls_container, champ_container, scattfig])
display(full_plot)

Input desired champion info in the format "TIER,NUMBER OF CLAIMED CHAMPS,NUMBER OF OTHER CLAIMED CHAMPS IN TIER"


    'data': [{'hoverinfo': 'y+name',
              'marker': {'color': 'Red', 'l…

# Alternative displays

In [560]:
level_widget = widgets.IntText(
            value=1.0,
            description='Current Level='
    )

#TODO: make a custom widget that accepts a list of tiers
tiers1 = widgets.IntText(
            value=1.0,
            description='Number of Tier1 Champions='
    )

tiers2 = widgets.IntText(
            value=0.0,
            description='Number of Tier2 Champions='
    )

tiers3 = widgets.IntText(
            value=0.0,
            description='Number of Tier3 Champions='
    )

tiers4 = widgets.IntText(
            value=0.0,
            description='Number of Tier4 Champions='
    )

tiers5 = widgets.IntText(
            value=0.0,
            description='Number of Tier5 Champions='
    )

In [561]:
rolls = 10
initial_level = 1
initial_tiers = []
initial_tiers.extend([1 for x in range(tiers1.value)])
initial_tiers.extend([2 for x in range(tiers2.value)])
initial_tiers.extend([3 for x in range(tiers3.value)])
initial_tiers.extend([4 for x in range(tiers4.value)])
initial_tiers.extend([5 for x in range(tiers5.value)])

In [562]:
#TODO: plot probs of each individual hit, and probs of any hit
total_trace = go.Scatter(x=list(range(1,rolls)), 
                         y=multi_rolls_prob(initial_level, initial_tiers),
                         line={'width':4.0},
                         opacity=0.5,
                        name='Hit on anything')
trace1 = go.Scatter(x=list(range(1,rolls)), 
                    y=multi_rolls_prob(initial_level, [1]),
                   line={'color':'DarkGray'},
                    #opacity=0.9,
                    name='Tier1 hit'
                   )

#TODO: Create additional five traces
g = go.FigureWidget(data=[total_trace, trace1],
             layout=go.Layout(
             title='TFTdex',
                 height=500,
             yaxis={'range':[0,10],
                   'title':'Expected Number of Hits',
                   'nticks':20},
            xaxis={'title':'Number of Rolls'}
             ))

In [563]:
# Function to handle input from widgets and alter graph
def validate():
    if 1 <= level_widget.value <= 9:
        return True
    else:
        return False
    
def response(change):
    if validate():
        level = level_widget.value
        tiers = []
        if tiers1.value <= tiers_df['Unique Characters'][0]: tiers.extend([1]*tiers1.value)
        if tiers2.value <= tiers_df['Unique Characters'][1]: tiers.extend([2]*tiers2.value)
        if tiers3.value <= tiers_df['Unique Characters'][2]: tiers.extend([3]*tiers3.value)
        if tiers4.value <= tiers_df['Unique Characters'][3]: tiers.extend([4]*tiers4.value)
        if tiers5.value <= tiers_df['Unique Characters'][4]: tiers.extend([5]*tiers5.value)
        #print(tiers)

        updated_probs = multi_rolls_prob(level, tiers)
        with g.batch_update():
            g.data[0].y = updated_probs
            g.data[1].y = multi_rolls_prob(level, [1])

#level_widget.observe(response, names='value')
#tiers1.observe(response, names='value')
#tiers2.observe(response, names='value')
#tiers3.observe(response, names='value')
#tiers4.observe(response, names='value')
#tiers5.observe(response, names='value')
g.observe(response, names='value')

In [564]:
widgets.VBox([level_widget, tiers1, tiers2, tiers3, tiers4, tiers5, g])

# Calculate the probability of a hit of one unit at a given level

In [16]:
level=5
tier=3
print('{0:.4f}'.format(hit_prob(level,tier)))

0.1042


# Calculate the probability of a hit of multiple units at a given level

In [17]:
level=5
tiers=[3,2,2]
print('{0:.4f}'.format(multi_hits_prob(level, tiers)))

0.3958


# Calculate the mean & variance of hits given a certain number of rolls

In [84]:
level=5
tiers=[3,2,2]
rolls=6
one_roll_prob=multi_hits_prob(level,tiers)
mean, var, skew, kurt = stats.binom.stats(rolls, one_roll_prob, moments='mvsk')
print('mean = {0:.4f} \nvariance = {1:.4f}'.format(mean,var))

mean = 2.3750 
variance = 1.4349


# Show Geometric CDF

In [85]:
rolls=10
samp_prob=.12
stats.geom.cdf(list(range(1,rolls+1)),.1042)

array([0.1042    , 0.19754236, 0.28115845, 0.35606174, 0.4231601 ,
       0.48326682, 0.53711042, 0.58534351, 0.62855072, 0.66725573])

In [102]:


scattA = custom_scatter(level,rolls,1,'A')

fig = go.FigureWidget(data=[scattA],
             layout=go.Layout(
                 title='TFTdex (Patch {})'.format(patch),
                 yaxis={'hoverformat':'.4f'},
                 height=500,
             ))

fig

FigureWidget({
    'data': [{'hoverinfo': 'y+name',
              'marker': {'color': '#A0A094'},
            …

In [101]:
def bar_display(level,rolls,champ,champ_letter):
    complete_bar = go.Bar(x=list(range(1,rolls+1)), 
              y=multi_rolls_prob(level, [int(champ)], rolls=rolls),
              name='Champion {}'.format(champ_letter),
              text=['{0:.4f}'.format(x) for x in multi_rolls_prob(level, [int(champ)], rolls=rolls)],
              textposition='auto',
              hoverinfo='y+name',
              marker={'color':color_dict[int(champ)],
                     'line': {'width':1.5}
                     }
             )
    return complete_bar

class bar_display(go.Bar):
    def __init__(self,level,rolls,champ,champ_letter):
        self.level = level
        self.rolls = rolls
        self.champ = champ
        self.letter = champ_letter
    bar_obj = go.Bar(x=list(range(1,rolls+1)), 
              y=multi_rolls_prob(level, [int(champ)], rolls=rolls),
              name='Champion A',
              text=['{0:.4f}'.format(x) for x in multi_rolls_prob(level_widget.value, [int(champA.value)], rolls=initial_rolls)],
              textposition='auto',
              hoverinfo='y+name',
                
              marker={'color':color_dict[int(champA.value)],
                     'line': {'width':1.5}
                     }
             )

SyntaxError: invalid syntax (<ipython-input-101-3878c828f383>, line 27)

In [36]:
fig

FigureWidget({
    'data': [{'marker': {'color': 'Red'},
              'type': 'scatter',
              'uid':…